In [630]:
#suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import pickle
# Import the relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

z_file = "./The_Zodiac_Race_Challenge_Dataset.csv"
z_df = pd.read_csv(z_file)

# Remove wrong col
z_df.drop('Had_A_Good_Sleep', axis=1, inplace=True)

X = z_df.iloc[:, :8]
Y = z_df.iloc[:, 8:]

animal_list = ['rat', 'ox', 'tiger', 'rabbit', 'dragon', 'snake', 
               'horse', 'goat', 'monkey', 'rooster', 'dog', 'pig']

top_3 = ['Winner', 'Second', 'Third']

#Generate df (one-hot encoded) indicating whether each animal (column) finishes in the top 3

Y_top3 = pd.DataFrame({animal:z_df[top_3].isin([animal]).apply(sum, axis=1) for animal in animal_list}).astype(bool)

# Data preprocessing

# Engineer Features
# Total distance:

X.eval('Total_distance = Desert + Mountain + Water + Land + Forest + Snow', inplace= True)

X['Is_Raining'] = X['Is_Raining'].map({'Yes':1, 'No':0}) 



In [631]:
# scale data
X_scaled = (X - X.min()) / (X.max() - X.min())
# X_scaled = (X - X.mean())/X.std()

In [632]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import copy

In [633]:
def get_time(top3_series, animal):

# INPUTS
# animal: name of animal
# top3_series: 6-entry series which represent top 3 animals and their finishing times: 
# [Winner, Winner_Time, Second, Second_Time ,Third, Third_Time]
# *Each row of Y is such a series

# OUTPUTS
# finishing time


    bool_arr = (top3_series==animal)
    
    map_dict = { 'Winner': 'Winner_Time',
                 'Second': 'Second_Time',
                 'Third' : 'Third_Time'
                }
    
    if any(bool_arr):

        return top3_series[map_dict[(top3_series==animal).idxmax()]]
     
    else:
        return np.nan

In [634]:
regen_timings = False

if regen_timings:
    
    Y_time = pd.DataFrame([[get_time(row[1], animal) for animal in animal_list] for row in Y.iterrows()],
                              columns= animal_list)
    
    pickle.dump(Y_time, open("Y_time", 'wb') )

else:
    Y_time= pickle.load(open("Y_time", 'rb'))

In [635]:
#1st stage:
#Predict whether each animal finishes in top 3


#split into training and test sets
X_train, X_test, Y_top3_train, Y_top3_test, Y_time_train, Y_time_test = train_test_split(X_scaled, Y_top3.astype(int), Y_time, random_state=0)

In [636]:
def acc_top3(clf_fitted, X_test, y_test):
    
#get accurracy of predicting top3 finishes (how well does classifier predict a true "top 3" label?)
#leaving out the majority of instances without a "top3" label
#clf: FITTED classifier

    y_test_intop3 = y_test[y_test==1]
    X_test_intop3 = X_test[y_test==1]
    
    y_pred_intop3= clf_fitted.predict(X_test_intop3)
    
    return y_pred_intop3.mean()

In [637]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(random_state=0),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
#     QuadraticDiscriminantAnalysis(),
    LogisticRegression()
    ]


clf_names = [clf.__class__.__name__ for clf in classifiers]

In [638]:
refit_clfs = False

def helper(clf, animal, X_train, Y_top3_train):
    
    print('fitting {} to {} \n'.format(clf.__class__.__name__, animal))
    return clf.fit(X_train, Y_top3_train[animal])

if refit_clfs==True:
    clfs_fitted_df = pd.DataFrame([[helper(clf, animal, X_train, Y_top3_train) for clf in copy.deepcopy(classifiers)] 
                                    for animal in animal_list],
                                  columns= clf_names,
                                  index = animal_list)
    pickle.dump(clfs_fitted_df, open("clfs_fitted_df", 'wb') )

else:
    clfs_fitted_df= pickle.load(open("clfs_fitted_df", 'rb'))


fitting KNeighborsClassifier to rat 

fitting SVC to rat 

fitting DecisionTreeClassifier to rat 

fitting RandomForestClassifier to rat 

fitting AdaBoostClassifier to rat 

fitting GradientBoostingClassifier to rat 

fitting GaussianNB to rat 

fitting LinearDiscriminantAnalysis to rat 

fitting LogisticRegression to rat 

fitting KNeighborsClassifier to ox 

fitting SVC to ox 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


fitting DecisionTreeClassifier to ox 

fitting RandomForestClassifier to ox 

fitting AdaBoostClassifier to ox 

fitting GradientBoostingClassifier to ox 

fitting GaussianNB to ox 

fitting LinearDiscriminantAnalysis to ox 

fitting LogisticRegression to ox 

fitting KNeighborsClassifier to tiger 

fitting SVC to tiger 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


fitting DecisionTreeClassifier to tiger 

fitting RandomForestClassifier to tiger 

fitting AdaBoostClassifier to tiger 

fitting GradientBoostingClassifier to tiger 

fitting GaussianNB to tiger 

fitting LinearDiscriminantAnalysis to tiger 

fitting LogisticRegression to tiger 

fitting KNeighborsClassifier to rabbit 

fitting SVC to rabbit 



...

C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


fitting DecisionTreeClassifier to dog 

fitting RandomForestClassifier to dog 

fitting AdaBoostClassifier to dog 

fitting GradientBoostingClassifier to dog 

fitting GaussianNB to dog 

fitting LinearDiscriminantAnalysis to dog 

fitting LogisticRegression to dog 

fitting KNeighborsClassifier to pig 

fitting SVC to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


fitting DecisionTreeClassifier to pig 

fitting RandomForestClassifier to pig 

fitting AdaBoostClassifier to pig 

fitting GradientBoostingClassifier to pig 

fitting GaussianNB to pig 

fitting LinearDiscriminantAnalysis to pig 

fitting LogisticRegression to pig 



C:\Users\Ivan\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [639]:
def compare_clfs(animal):

#given name of animal, generates dataframe comparing classifiers 

    return pd.DataFrame([(clf_fitted.__class__.__name__
                          ,clf_fitted.score(X_test, Y_top3_test[animal])
#                           ,acc_top3(clf_fitted, X_test, Y_top3_test[animal])
                         ) for clf_fitted in clfs_fitted_df.loc[animal]]
                           ,columns=['classifier', 'acc']).set_index('classifier')

In [640]:
clf_all_animals = pd.concat([compare_clfs(animal) for animal in animal_list], axis=1, keys=animal_list)
# clf_all_animals.reindex((df.columns), axis=1)
clf_all_animals

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc,acc
classifier,,,,,,,,,,,,
KNeighborsClassifier,0.8972,0.9364,0.9082,0.9409,0.9247,0.8777,0.9061,0.9166,0.9053,0.9475,0.9036,0.9273
SVC,0.9022,0.9571,0.9084,0.9555,0.9291,0.8768,0.8813,0.9111,0.8817,0.9562,0.8977,0.9238
DecisionTreeClassifier,0.9238,0.9833,0.9208,0.9795,0.9241,0.9065,0.9197,0.9245,0.9173,0.9569,0.9272,0.9588
RandomForestClassifier,0.9456,0.9868,0.9421,0.9854,0.9484,0.9266,0.9465,0.9460,0.9438,0.9688,0.9483,0.9711
AdaBoostClassifier,0.9242,0.9783,0.9356,0.9786,0.9273,0.8959,0.9112,0.9298,0.8920,0.9634,0.9238,0.9546
GradientBoostingClassifier,0.9450,0.9851,0.9423,0.9852,0.9550,0.9241,0.9459,0.9493,0.9340,0.9695,0.9453,0.9667
GaussianNB,0.8443,0.9502,0.8654,0.9471,0.8960,0.8157,0.8630,0.8792,0.8481,0.9496,0.8519,0.8820
LinearDiscriminantAnalysis,0.7804,0.9445,0.8348,0.9379,0.9125,0.7187,0.8676,0.9000,0.8527,0.9441,0.7933,0.8690


In [641]:
# clf_all_animals=clf_all_animals.swaplevel(axis=1)
# clf_all_animals.reindex(sorted(clf_all_animals.columns), axis=1)
clf_all_animals.loc['best_clfs'] = clf_all_animals.idxmax()

name_clf_dict = {clf.__class__.__name__:clf for clf in classifiers}
clf_all_animals.loc['best_clfs'].value_counts()

best_clf = name_clf_dict[clf_all_animals.loc['best_clfs'].mode().loc[0]]
print('best clf: {}'.format(best_clf.__class__.__name__))

best clf: RandomForestClassifier


In [642]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression

In [643]:
regressors = [
#               GradientBoostingRegressor(),
              GradientBoostingRegressor(n_estimators=1000, max_depth=5),
              RandomForestRegressor(), 
              AdaBoostRegressor(),
              LogisticRegression()
             ]

rgg_names = [rgg.__class__.__name__ for rgg in regressors]

In [665]:
refit_rggs = True

def helper_rgg(rgg, animal, X_train, Y_time_train):
    
    X_selected = X_train[~Y_time_train[animal].isnull()]
    y_selected = Y_time_train[animal][~Y_time_train[animal].isnull()]

    print('fitting {} to {} \n'.format(rgg.__class__.__name__, animal))
    return rgg.fit(X_selected, y_selected)

if refit_rggs==True:
    rggs_fitted_df = pd.DataFrame([[helper_rgg(rgg, animal, X_train, Y_time_train) for rgg in copy.deepcopy(regressors)] 
                                    for animal in animal_list]
                                   ,columns= rgg_names
                                   ,index = animal_list)
    pickle.dump(rggs_fitted_df, open("rggs_fitted_df", 'wb') )

else:
    rggs_fitted_df= pickle.load(open("rggs_fitted_df", 'rb'))


fitting GradientBoostingRegressor to rat 

fitting RandomForestRegressor to rat 

fitting AdaBoostRegressor to rat 

fitting LogisticRegression to rat 

fitting GradientBoostingRegressor to ox 

fitting RandomForestRegressor to ox 

fitting AdaBoostRegressor to ox 

fitting LogisticRegression to ox 

fitting GradientBoostingRegressor to tiger 

fitting RandomForestRegressor to tiger 

fitting AdaBoostRegressor to tiger 

fitting LogisticRegression to tiger 

fitting GradientBoostingRegressor to rabbit 

fitting RandomForestRegressor to rabbit 

fitting AdaBoostRegressor to rabbit 

fitting LogisticRegression to rabbit 

fitting GradientBoostingRegressor to dragon 

fitting RandomForestRegressor to dragon 

fitting AdaBoostRegressor to dragon 

fitting LogisticRegression to dragon 

fitting GradientBoostingRegressor to snake 

fitting RandomForestRegressor to snake 

fitting AdaBoostRegressor to snake 

fitting LogisticRegression to snake 

fitting GradientBoostingRegressor to horse 

f

In [666]:
rggs_fitted_df

,GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,LogisticRegression
rat,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
ox,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
tiger,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
rabbit,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
dragon,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
snake,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
horse,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
goat,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
monkey,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."
rooster,([DecisionTreeRegressor(criterion='friedman_ms...,"(DecisionTreeRegressor(criterion='mse', max_de...","(DecisionTreeRegressor(criterion='mse', max_de...","LogisticRegression(C=1.0, class_weight=None, d..."


In [667]:
def compare_rggs(animal):

#given name of animal, generates dataframe comparing classifiers 

    return pd.DataFrame([(rgg_fitted.__class__.__name__
                          ,rgg_fitted.score(X_train[~Y_time_train[animal].isnull()], 
                                            Y_time_train[animal][~Y_time_train[animal].isnull()])
                         ) for rgg_fitted in rggs_fitted_df.loc[animal]]
                           ,columns=['regressor', 'acc']).set_index('regressor')

In [668]:
rgg_all_animals = pd.concat([compare_rggs(animal) for animal in animal_list], axis=1, keys=animal_list)
rgg_all_animals.loc['best_rggs'] = rgg_all_animals.idxmax()

In [669]:
name_rgg_dict = {rgg.__class__.__name__:rgg for rgg in regressors}
rgg_all_animals.loc['best_rggs'].value_counts()

best_rgg = name_rgg_dict[rgg_all_animals.loc['best_rggs'].mode().loc[0]]
print('best rgg: {}'.format(best_rgg.__class__.__name__))

best rgg: GradientBoostingRegressor


In [649]:
# Predict top3 using regression
# Compare results vs using classifer technique to predict top 3

def pred_time(X, best_rgg):
# rgg_fitted_series: series of fitted (single) regressor for all all animals 
# returns dataframe Y predicted timings
    return pd.DataFrame([rggs_fitted_df.loc[animal,best_rgg.__class__.__name__].predict(X) for animal in animal_list], 
                        index=animal_list,
                        columns=X.index
                       ).T
    
    

In [650]:
def pred_top3_rgg(X, best_rgg):
# returns dataframe Y of (one-hot) indicator of whether animal finished in top3
    return pred_time(X_test, best_rgg).apply(lambda row: row.nsmallest(3) ,axis=1).notnull().reindex(animal_list, axis=1)
    

In [651]:
def pred_top3_clf(X, best_clf):
# returns dataframe Y of (one-hot) indicator of whether animal finished in top3
    return pd.DataFrame([clfs_fitted_df.loc[animal,best_clf.__class__.__name__]
                         .predict(X) for animal in animal_list],
                        index=animal_list,
                        columns=X.index
                       ).T

In [652]:
pred_top3_clf_df = pred_top3_clf(X_test, best_clf)

In [653]:
pred_top3_rgg_df= pred_top3_rgg(X_test,best_rgg)

In [654]:
print("Accuracy in predicting whether animal finishes in top3 \nUsing Regression: {} \nUsing Classification: {}" \
      .format((Y_top3_test == pred_top3_rgg_df).mean().mean(),(Y_top3_test == pred_top3_clf_df).mean().mean())
     )

Accuracy in predicting whether animal finishes in top3 
Using Regression: 0.7658 
Using Classification: 0.9549499999999999


In [655]:
# Predict top 3 finihers in order using

# Method 1
# Pure regression: regression, then sort by timings

# Method 2
# Classification to pick animals predicted to finish in top3, then sort by timings 
# If < 3 finishers are predicted to be in top3, choose next fastest timing from animal not predicted in top3

In [656]:
#Generate test rank set, entries where animals are tied

Y_rank_test = Y_time_test.rank(axis=1)
X_rank_test = X_test

In [657]:
# # Remove entries where 2 or more of the top3 finishers are tied
# tied = Y_rank_test.mode(axis=1)[2].isnull()

# Y_rank_test = Y_rank_test[~tied]
# X_rank_test = X_test[~tied]

In [658]:
#Method 1

#Dataframe of predicting rankings using regression
pred_rank_rgg = pred_time(X_rank_test, best_rgg).rank(axis=1).apply(lambda row: row.nsmallest(3) ,axis=1).reindex(animal_list, axis=1)

#number of positions predicted correctly for each entry
print((pred_rank_rgg==Y_rank_test).sum(axis=1).value_counts())

#proportion of times where all top3 rankings where predicted correctly
rank_acc_rgg = ((pred_rank_rgg==Y_rank_test).sum(axis=1)==3).mean()

0    5552
1    2086
3    1358
2    1004
dtype: int64


In [659]:
#Method 2

def pred_rank_m2(X, best_rgg, best_clf):
    pred_rank = pred_time(X, best_rgg).rank(axis=1)
    pred_top3_clf_df = pred_top3_clf(X, best_clf)
    
    #ranking(using timings from rgg) of animals predicted by clf to finish in top 3 
    intop3_ranked = pred_rank[pred_top3_clf_df.astype(bool)]
    outtop3_ranked = pred_rank[~pred_top3_clf_df.astype(bool)] + 12
    
    return  intop3_ranked.fillna(outtop3_ranked).apply(lambda row: row.nsmallest(3) ,axis=1) \
            .reindex(animal_list, axis=1).rank(axis=1)


In [660]:
pred_rank_m2_df = pred_rank_m2(X_rank_test, best_rgg, best_clf)

In [661]:
print((pred_rank_m2_df==Y_rank_test).sum(axis=1).value_counts())

3    3971
1    2953
2    2087
0     989
dtype: int64


In [662]:
rank_acc_m2 = ((pred_rank_m2_df==Y_rank_test).sum(axis=1)==3).mean()

In [674]:
clfs_fitted_df = pd.DataFrame([[helper(clf, animal, X_scaled, Y_top3) for clf in copy.deepcopy([best_clf])] 
                                for animal in animal_list],
                              columns= [best_clf.__class__.__name__],
                              index = animal_list)

rggs_fitted_df = pd.DataFrame([[helper_rgg(rgg, animal, X_scaled, Y_time) for rgg in copy.deepcopy([best_rgg])] 
                                for animal in animal_list]
                               ,columns= [best_rgg.__class__.__name__]
                               ,index = animal_list)



fitting RandomForestClassifier to rat 

fitting RandomForestClassifier to ox 

fitting RandomForestClassifier to tiger 

fitting RandomForestClassifier to rabbit 

fitting RandomForestClassifier to dragon 

fitting RandomForestClassifier to snake 

fitting RandomForestClassifier to horse 

fitting RandomForestClassifier to goat 

fitting RandomForestClassifier to monkey 

fitting RandomForestClassifier to rooster 

fitting RandomForestClassifier to dog 

fitting RandomForestClassifier to pig 

fitting GradientBoostingRegressor to rat 

fitting GradientBoostingRegressor to ox 

fitting GradientBoostingRegressor to tiger 

fitting GradientBoostingRegressor to rabbit 

fitting GradientBoostingRegressor to dragon 

fitting GradientBoostingRegressor to snake 

fitting GradientBoostingRegressor to horse 

fitting GradientBoostingRegressor to goat 

fitting GradientBoostingRegressor to monkey 

fitting GradientBoostingRegressor to rooster 

fitting GradientBoostingRegressor to dog 

fitting G

In [676]:
X_submit = pd.DataFrame([[2345, 8764, 6689, 7332, 1050, 3741, 40]], columns=X_test.columns[:7])
X_submit['Is_Raining']=0
X_submit['Total_distance']= X_submit.loc[::7].sum(axis=1)

pred_rank_m2(X_submit, best_rgg, best_clf)

,rat,ox,tiger,rabbit,dragon,snake,horse,goat,monkey,rooster,dog,pig
0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
